In [ ]:
#unzip BSD datasets
!unzip '/content/drive/MyDrive/KGT/KGT_validation.zip' -d '/content'
!unzip '/content/drive/MyDrive/KGT/KGT_test.zip' -d '/content'
!unzip '/content/drive/MyDrive/KGT/100_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/KGT/50_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/KGT/20_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/KGT/10_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/KGT/5_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/KGT/3_samples.zip' -d '/content'


"\n!unzip '/content/drive/MyDrive/KGT/KGT_validation.zip' -d '/content'\n!unzip '/content/drive/MyDrive/KGT/KGT_test.zip' -d '/content'\n!unzip '/content/drive/MyDrive/KGT/100_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/KGT/50_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/KGT/20_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/KGT/10_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/KGT/5_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/KGT/3_samples.zip' -d '/content'\n"

In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
import imgaug.augmenters as iaa
import imgaug as ia
import os

In [ ]:
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:
#load BSD images
def load_data(dir_path, labels_dict):
  all_names = os.listdir(dir_path)
  data_x = []
  data_y = []
  for name in all_names:
    img = keras.preprocessing.image.load_img(os.path.join(dir_path, name), target_size=(100,100))
    img = keras.preprocessing.image.img_to_array(img,dtype='uint8')
    data_x.append(img)
    lab = labels_dict[name[:1]]
    data_y.append(lab)
  data_x = np.array(data_x)
  data_y = np.array(data_y)
  return data_x, data_y
class BSDdata(keras.utils.Sequence):
    def __init__(self, data_x, data_y, batch_size, train):
      self.data_x = data_x
      self.data_y = data_y
      self.train = train
      self.batch_size = batch_size

    def __len__(self):
      return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):
      batch_x = self.data_x[idx * self.batch_size:(idx + 1) * self.batch_size]
                            
      batch_y = self.data_y[idx * self.batch_size:(idx + 1) * self.batch_size]
      if self.train == True:
          batch_x = Data_Aug(batch_x)

      batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
      batch_y = keras.utils.to_categorical(batch_y, 2)
      return batch_x, batch_y

In [ ]:
#save the best model during training
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        #self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
def create_AlexNet(input_shape, num_classes=6, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(96, (3,3), strides=(2,2), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
  x = keras.layers.Conv2D(256, (3,3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x) 
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
 
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  feature_maps = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
  x = keras.layers.Flatten()(feature_maps)
  #x = keras.layers.Dense(256,activation='relu')(x)
  #x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(128,activation='relu')(x)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
def create_VGGNet(input_shape, num_classes,  random_seed=None):
  inputs = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(64, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(64, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.2)(x)

  x = keras.layers.Conv2D(128, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(128, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.3)(x)
  
  x = keras.layers.Conv2D(256, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(256, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(256, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.4)(x)


    
  x = keras.layers.Conv2D(512, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(512, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(512, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  p3 = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.5)(p3)


 
   
  '''  
  x = keras.layers.Conv2D(512, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(512, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(512, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.5)(x) 
  '''
  
 

  #x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(128,  kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
  return keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
def create_ResNet(input_shape, num_classes=6, num_filters = 64, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x)

  
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(2*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 

    
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(4*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 

   
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(8*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 
  
  
  x = keras.layers.GlobalAveragePooling2D()(x)
  outputs = keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
#load training, validation and testing images
training_data_path = '50_samples'
validation_data_path = 'KGT_validation'
test_data_path = 'KGT_test'
labels_dict = {'N': 0, 'P': 1}
x_train, y_train = load_data(training_data_path, labels_dict)
x_val, y_val = load_data(validation_data_path, labels_dict)
x_test, y_test = load_data(test_data_path, labels_dict)
_, x_val, _, y_val = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=2000,
    random_state=0,
    stratify=y_val)
x_test = x_test/255.0

In [ ]:
#construct model
input_size = (100, 100, 3)

model = create_AlexNet(input_shape=input_size, num_classes=2, random_seed=None)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                    loss=['categorical_crossentropy'],
                    metrics=['accuracy'])

train_sequence = BSDdata(x_train, y_train, batch_size=10, train=True)
validation_sequence = BSDdata(x_val, y_val, batch_size=10, train=False)

#training
history = model.fit(
    train_sequence,
    epochs=300,
    validation_data=validation_sequence,
    #validation_steps=12,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='100_samples_seed_fix.h5',
        patience=10))

Epoch 1/300
10/10 [==============================] - 4s 164ms/step - loss: 0.7718 - accuracy: 0.7500 - val_loss: 1.9635 - val_accuracy: 0.5040
Epoch 2/300
10/10 [==============================] - 1s 139ms/step - loss: 0.6683 - accuracy: 0.8100 - val_loss: 0.9249 - val_accuracy: 0.4840
Epoch 3/300
10/10 [==============================] - 1s 137ms/step - loss: 0.3701 - accuracy: 0.8500 - val_loss: 0.8616 - val_accuracy: 0.4785
Epoch 4/300
10/10 [==============================] - 1s 154ms/step - loss: 0.2396 - accuracy: 0.9000 - val_loss: 0.9748 - val_accuracy: 0.4880
Epoch 5/300
10/10 [==============================] - 1s 139ms/step - loss: 0.1177 - accuracy: 0.9600 - val_loss: 0.8165 - val_accuracy: 0.4110
Epoch 6/300
10/10 [==============================] - 1s 137ms/step - loss: 0.1997 - accuracy: 0.9100 - val_loss: 0.7994 - val_accuracy: 0.4560
Epoch 7/300
10/10 [==============================] - 1s 136ms/step - loss: 0.1449 - accuracy: 0.9500 - val_loss: 0.8412 - val_accuracy: 0.4250

In [ ]:
evaluate1 = model.evaluate(x_test, keras.utils.to_categorical(y_test,2))
evaluate1

128/128 [==============================] - 1s 8ms/step - loss: 0.7292 - accuracy: 0.9013


[0.7291783094406128, 0.9012770056724548]

In [ ]:
model_train = keras.models.load_model('100_samples_seed_fix.h5')
evaluate2 = model_train.evaluate(x_test, keras.utils.to_categorical(y_test,2))
evaluate2

128/128 [==============================] - 1s 7ms/step - loss: 0.7292 - accuracy: 0.9013


[0.7291783094406128, 0.9012770056724548]